In [ ]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.coils as jsc

from tests.coils.test_coils import _check_single_vectorized
import pyvista as pv
from dataclasses import dataclass

from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[0]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

In [ ]:
class bb:
    param : str
    def __init__(self, param):
        self.param = param

def _get_all_discrete_coils(request):    
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)    
    coilset_jaxsbgeom = [jsb.coils.DiscreteCoil.from_positions(coilset_sbgeom[i].Get_Vertices()) for i in range(coilset_sbgeom.Number_of_Coils())]
    
    return coilset_jaxsbgeom, coilset_sbgeom

def _get_all_fourier_coils_trunc(request):
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)
    coilset_fourier = SBGeom.Coils.Convert_to_Fourier_Coils(coilset_sbgeom, Nftrunc = 20)
    
    coilset_jax = [jsb.coils.FourierCoil(jnp.array(i.Get_Fourier_Cos()), jnp.array(i.Get_Fourier_Sin()), jnp.array(i.Get_Centre())) for i in coilset_fourier]
    return coilset_jax, coilset_fourier
    

dcoils_jax, coils_sbgeom = _get_all_discrete_coils(bb(coil_files[0]))
coils_jax, coils_sbgeom = _get_all_fourier_coils_trunc(bb(coil_files[0]))

In [ ]:
jsc.CoilSet.from_list(dcoils_jax[0:2] + coils_jax[0:1]).position(jnp.array([0.0, 1.0]))

In [ ]:
def check_vector_coilset(coils_jax):
    s =  jnp.linspace(0, 1, 1000)

    pos_base = []
    tan_base = []

    for i in coils_jax:
        pos_base.append(i.position(s))
        tan_base.append(i.tangent(s))

    pos_base = jnp.array(pos_base)
    tan_base = jnp.array(tan_base)

    coilsetv = jsb.coils.CoilSet.from_list(coils_jax)


    pos_vec = coilsetv.position(s)
    tan_vec = coilsetv.tangent(s)

    onp.testing.assert_allclose(pos_base, pos_vec)
    onp.testing.assert_allclose(tan_base, tan_vec)


check_vector_coilset(dcoils_jax)
    

In [ ]:
coilset_v = jsb.coils.CoilSet.from_list(coils_jax)

In [ ]:
def coil_position(coil, s):
    return coil.position(s)



jax.vmap(coil_position, in_axes=(0, None))(coilset_v.coils, jnp.linspace(0, 1, 5))

In [ ]:
coilset_v.coils.fourier_cos.shape[:-2]

In [ ]:
coilset_v.position(0.0)